In [6]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from pydataset import data
import re
import nltk

%matplotlib inline

In [7]:
df = pd.read_csv("data/spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Ci..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 t...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [8]:
df.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [27]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [28]:
df['cleanText'] = df['Message'].apply(lambda x: normalize_document(x))
df.head()

,Category,Message,cleanText
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Ci...",go jurong point crazy available bugis n great world la e buffet cine got amore wat
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 t...,free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives around here though",nah dont think goes usf lives around though


In [31]:
import collections
results = collections.Counter()
df['cleanText'].str.split().apply(results.update)
for word, count in results.most_common(10):
    print(word,": ",count)

u :  1130
call :  575
2 :  482
im :  473
ur :  390
get :  386
dont :  298
4 :  293
go :  281
ok :  278


# Bag of Words Model


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(df['cleanText'])
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
vocab = cv.get_feature_names()
pd.DataFrame(cv_matrix, columns=vocab)

,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,02070836089,...,zebra,zed,zeros,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Bag of N-Grams Model

In [34]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(df['cleanText'])
bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

,008704050406 sp,0089my last,0121 2025050,01223585236 xx,01223585334 cum,0125698789 ring,02 user,020603 2nd,0207 153,02072069400 bx,...,zed pobox,zeros savings,zhong se,zindgi wo,zoe 18,zoe hit,zogtorius ive,zoom cine,zouk nichols,zyada kisi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# TF - IDF

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(df['cleanText'])
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,02070836089,...,zebra,zed,zeros,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# document similary via cosine

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,5562,5563,5564,5565,5566,5567,5568,5569,5570,5571
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.036959,0.000000,0.000000,0.0,0.000000,0.000000
1,0.000000,1.000000,0.000000,0.000000,0.000000,0.040733,0.000000,0.000000,0.000000,0.000000,...,0.047576,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034093,...,0.000000,0.000000,0.000000,0.0,0.039992,0.000000,0.000000,0.0,0.020001,0.000000
3,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.137548,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.060819,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.040733,0.000000,0.000000,0.000000,1.000000,0.066157,0.000000,0.000000,0.000000,...,0.025823,0.060000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.083140,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.066157,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.164760,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.073823,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.035963,0.000000,0.0,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.013614,...,0.000000,0.000000,0.000000,0.0,0.070544,0.117038,0.000000,0.0,0.000000,0.000000
9,0.000000,0.000000,0.034093,0.000000,0.000000,0.000000,0.000000,0.000000,0.013614,1.000000,...,0.000000,0.000000,0.000000,0.0,0.063419,0.015250,0.000000,0.0,0.051458,0.000000


In [ ]:
# cluster via document similarity

In [38]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=25)
km.fit_transform(similarity_df)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([df, cluster_labels], axis=1)

,Category,Message,cleanText,ClusterLabel
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Ci...",go jurong point crazy available bugis n great world la e buffet cine got amore wat,19
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,10
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 t...,free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry...,6
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,6
4,ham,"Nah I don't think he goes to usf, he lives around here though",nah dont think goes usf lives around though,14
5,spam,FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun y...,freemsg hey darling 3 weeks word back id like fun still tb ok xxx std chgs send 150 rcv,6
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,even brother like speak treat like aids patent,6
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as...,per request melle melle oru minnaminunginte nurungu vettam set callertune callers pres...,6
8,spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize re...,winner valued network customer selected receivea 900 prize reward claim call 090617014...,22
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles...,mobile 11 months u r entitled update latest colour mobiles camera free call mobile upd...,8


In [ ]:
# Bi-grams

In [40]:
bigram_converter = CountVectorizer(ngram_range=(2,2), token_pattern='(?u)\\b\\w+\\b')
x2 = bigram_converter.fit_transform(df['cleanText'])

In [45]:
bigrams = bigram_converter.get_feature_names()
print(len(bigrams))
bigrams[-10:]

33372


['zed pobox',
 'zeros savings',
 'zhong se',
 'zindgi wo',
 'zoe 18',
 'zoe hit',
 'zogtorius ive',
 'zoom cine',
 'zouk nichols',
 'zyada kisi']